### Обучение TextToLabel

In [4]:
import pandas as pd

files: list[str] = [
    "dataset/transcription/transcribed.csv",
    # "dataset/transcription/hr_bot_clear.json",
    # "dataset/transcription/hr_bot_noise.json",
    # "dataset/transcription/hr_bot_synt.json",
    # "dataset/transcription/luga.json",
]

li = []
for filename in files:
    df = pd.read_csv(filename)
    li.append(df)
df = pd.concat(li, axis=0, ignore_index=True)
df = df.sample(frac=1)
df.head()

,Unnamed: 0,audio_filepath,id,text,label,attribute,transcription
189,1375,dataset/hr_bot_clear/65271963-76ff-11ee-9ba3-c...,65271963-76ff-11ee-9ba3-c09bf4619c03,осадить на шесть вагонов,4,6,а садить на шесть вагонов
1809,2498,dataset/hr_bot_clear/58b9e1ef-76ff-11ee-b5c5-c...,58b9e1ef-76ff-11ee-b5c5-c09bf4619c03,осадить на девять вагонов,4,9,осадить на девять вагонов
2555,1639,dataset/hr_bot_clear/428d29f4-76ff-11ee-9499-c...,428d29f4-76ff-11ee-9499-c09bf4619c03,протянуть на пятнадцать вагонов,10,15,протянуть на пятнадцать вагонов
857,632,dataset/hr_bot_noise/2f1d7d35-76fe-11ee-ade9-c...,2f1d7d35-76fe-11ee-ade9-c09bf4619c03,сжать автосцепки,16,-1,сжате напяня зетка
1446,2062,dataset/hr_bot_clear/80b1fc0c-76ff-11ee-9dde-c...,80b1fc0c-76ff-11ee-9dde-c09bf4619c03,протянуть на два вагона,10,2,протянуть нам два вагона


In [5]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

vectorizer = TfidfVectorizer(
    lowercase=True, sublinear_tf=True, max_df=0.5, min_df=5, stop_words="english"
)
classifier = RandomForestClassifier(n_estimators=100, random_state=42)

pipeline = Pipeline(
    [
        ("tfidf", vectorizer),
        ("classifier", classifier),
    ]
)

In [7]:
from label2id import id2label

X = df["transcription"]
y = df["label"].apply(id2label).values

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline.fit(X_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.5, min_df=5, stop_words='english',
                                 sublinear_tf=True)),
                ('classifier', RandomForestClassifier(random_state=42))])

In [9]:
from sklearn.model_selection import cross_val_score

score = pipeline.score(X_test, y_test)
print('Точность на тестовом наборе:', score)

# Выполните перекрестную проверку для более надежной оценки
scores = cross_val_score(pipeline, X, y, cv=5)
print('Средняя точность по перекрестной проверке:', scores.mean())

Точность на тестовом наборе: 0.8870056497175142
Средняя точность по перекрестной проверке: 0.8994350282485876


In [10]:
import joblib
joblib.dump(pipeline, "trained/text2label.pkl")

['trained/text2label.pkl']